<a href="https://colab.research.google.com/github/parus-cristatus/tolokapizza/blob/main/tolokaapi/general_tasks_upload_aiohttp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Used Toloka preset *Image classification* . The template contains a single and relatively short input field called *image* (image link)
The script won't work properly in the Sandbox, there will be *TOO_MANY_REQUESTS* errors

In [ ]:
!pip install nest-asyncio

In [ ]:
import csv
import json
import time
import asyncio
import aiohttp
import nest_asyncio
from typing import Generator, Any
from aiohttp.client import ClientSession

In [ ]:
token = 'y0_AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
target = 'PRODUCTION' 

In [ ]:
pool = '36781956'
api_resource = 'https://toloka.dev/api/v1/tasks' # PRODUCTION

In [ ]:
# create 10K test tasks in csv file to read from it
for _ in range(10000):
    with open('data.csv', mode='a', encoding='utf-8') as f:
        w = csv.writer(f, lineterminator='\n')
        w.writerow(['https://wallpaperaccess.com/full/4023744.jpg'])

In [ ]:
with open('data.csv', 'r') as f:
    dataset = [i[0] for i in csv.reader(f)]

tasks = [
    {"pool_id": pool, "input_values": {"image": url}, "overlap": 1, "infinite_overlap": False}
    for url in dataset
    ]

In [ ]:
nest_asyncio.apply()

In [ ]:
async def tasks_aiter(tasks: list) -> Generator[Any, None, None]:
    for task in tasks:
        yield task

async def upload_tasks(toloka_tasks: list, token: str) -> None:
    headers = {
        "Authorization": f"OAuth {token}",
        "Content-Type": "application/JSON"
    }
    async with aiohttp.ClientSession(headers=headers) as session:
        tasks = []
        async for data in tasks_aiter(toloka_tasks):
            tasks.append(post_call(session, data))
        await asyncio.gather(*tasks)

async def post_call(session: ClientSession, data: dict) -> None:
    url = api_resource
    async with session.post(url, json=data) as response:
        await response.json(content_type=None)

In [ ]:
start = time.time()
asyncio.run(upload_tasks(tasks, token))
print(round(time.time() - start, 3))